In [1]:
#
# !pip install pandas as pd

In [2]:
import re
import numpy as np
import pandas as pd
import logging
import warnings
warnings.filterwarnings('ignore')
import unicodedata
from datetime import datetime
import os

In [3]:
# Directorio donde se encuentran los archivos
DIRECTORIO_SAFETRACK = "../Safetrack"

# Patrón de regex para coincidir con los nombres de archivos esperados
PATRON_ARCHIVO = r"Reporte de viaje\((\d{8})-(\d{8})\)\.xlsx$"

# Mapeo de números de mes a nombres en español
MESES_ES = [
    'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
    'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'
]

def listar_archivos(directorio, patron):
    """
    Lista todos los archivos en el directorio que coinciden con el patrón dado.
    
    :param directorio: Ruta al directorio donde buscar los archivos.
    :param patron: Expresión regular para filtrar los archivos.
    :return: Lista de rutas completas a los archivos que coinciden.
    """
    archivos_coincidentes = []
    for archivo in os.listdir(directorio):
        if re.match(patron, archivo):
            ruta_completa = os.path.join(directorio, archivo)
            archivos_coincidentes.append(ruta_completa)
    return archivos_coincidentes

def extraer_mes(nombre_archivo):
    """
    Extrae el mes en español a partir del nombre del archivo.
    
    :param nombre_archivo: Nombre del archivo.
    :return: Nombre del mes en español.
    :raises ValueError: Si el formato del archivo no es válido.
    """
    match = re.match(PATRON_ARCHIVO, nombre_archivo)
    if not match:
        raise ValueError(f"El archivo '{nombre_archivo}' no coincide con el patrón esperado.")
    
    fecha_inicio_str = match.group(1)  # 'YYYYMMDD'
    
    try:
        fecha_inicio = datetime.strptime(fecha_inicio_str, '%Y%m%d')
    except ValueError:
        raise ValueError(f"La fecha de inicio en '{nombre_archivo}' no es válida.")
    
    nombre_mes = MESES_ES[fecha_inicio.month - 1]
    return nombre_mes

def extraer_fecha_inicio(nombre_archivo):
    """
    Extrae la fecha de inicio como objeto datetime a partir del nombre del archivo.
    
    :param nombre_archivo: Nombre del archivo.
    :return: Objeto datetime de la fecha de inicio.
    """
    match = re.match(PATRON_ARCHIVO, nombre_archivo)
    if not match:
        raise ValueError(f"El archivo '{nombre_archivo}' no coincide con el patrón esperado.")
    
    fecha_inicio_str = match.group(1)
    fecha_inicio = datetime.strptime(fecha_inicio_str, '%Y%m%d')
    return fecha_inicio

def generar_listas(directorio):
    """
    Genera las listas de archivos y meses automáticamente.
    
    :param directorio: Ruta al directorio donde se encuentran los archivos.
    :return: Tuple (archivos, meses)
    """
    archivos = listar_archivos(directorio, PATRON_ARCHIVO)
    
    # Ordenar los archivos por fecha de inicio para mantener un orden lógico
    archivos.sort(key=lambda x: extraer_fecha_inicio(os.path.basename(x)))
    
    meses = [extraer_mes(os.path.basename(archivo)) for archivo in archivos]
    
    # (Opcional) Eliminar duplicados en 'meses' mientras se mantiene el orden
    meses_unicos = list(dict.fromkeys(meses))
    
    return archivos, meses_unicos

# Generar las listas de archivos y meses automáticamente
archivos, meses = generar_listas(DIRECTORIO_SAFETRACK)

# Inicializar lista_tracks como en tu código original
lista_tracks = []

# Imprimir las listas resultantes para verificar (opcional)
print("Archivos a procesar:")
for archivo in archivos:
    print(f" - {archivo}")

print("\nMeses extraídos:")
for mes in meses:
    print(f" - {mes}")

Archivos a procesar:
 - ../Safetrack\Reporte de viaje(20241101-20241130).xlsx
 - ../Safetrack\Reporte de viaje(20241201-20241231).xlsx
 - ../Safetrack\Reporte de viaje(20250101-20250131).xlsx
 - ../Safetrack\Reporte de viaje(20250201-20250212).xlsx

Meses extraídos:
 - Noviembre
 - Diciembre
 - Enero
 - Febrero


### Agregamos funciones de logging:

In [4]:
def log_proceso(df_antes, df_despues, etapa):
  eliminados = len(df_antes) - len(df_despues)
  porcentaje = (eliminados / len(df_antes)) * 100 if len(df_antes) > 0 else 0
  print(f"\n=== Etapa: {etapa} ===")
  print(f"Registros antes: {len(df_antes)}")
  print(f"Registros después: {len(df_despues)}")
  print(f"Registros eliminados: {eliminados} ({porcentaje:.2f}%)")

  if eliminados > 0:
      registros_eliminados = df_antes[~df_antes.index.isin(df_despues.index)]
      # Guardar registros eliminados en un archivo
      archivo_eliminados = f"../Limpia/eliminados_{etapa.lower().replace(' ', '_')}.xlsx"
      registros_eliminados.to_excel(archivo_eliminados, index=False)
      print(f"Registros eliminados guardados en: {archivo_eliminados}")

## Reporte de viaje Safetrack Uruguay -> estacionados_camion.xlsx

In [5]:
def limpiar_archivo(archivo):
    """Carga y limpia un archivo Excel.
    Args:
        archivo (str): Ruta al archivo Excel.
    Returns:
        pd.DataFrame: DataFrame con los datos limpios.
    """
    # Cargar el archivo
    df = pd.read_excel(archivo, engine="openpyxl")

    # Eliminar las primeras 3 filas y resetear el índice
    df = df.iloc[3:].reset_index(drop=True)

    # Renombrar las columnas
    df.columns = [
        'Indice', 'Numero_de_placa', 'Estado_de_viaje', 'Tiempo_de_Inicio',
        'Tiempo_Final', 'Kilometraje_km', 'Duracion', 'Lugar_de_inicio', 'Fin_Localizacion'
    ]

    # Convertir columnas de tiempo a formato datetime
    df['Tiempo_de_Inicio'] = pd.to_datetime(df['Tiempo_de_Inicio'], errors='coerce')
    df['Tiempo_Final'] = pd.to_datetime(df['Tiempo_Final'], errors='coerce')

    return df


estacionados_camion_total = []  # Lista para guardar los datos de todos los meses

for i, archivo in enumerate(archivos):
    track = limpiar_archivo(archivo)
    lista_tracks.append(track)
    
    # Filtrar registros donde el Estado_de_viaje sea "Estacionamiento"
    # estacionados_camion = track[track["Estado_de_viaje"] == "Estacionamiento"].copy()
    estacionados_camion = track.copy()
    
    # Eliminar columnas innecesarias
    estacionados_camion.drop(columns=["Fin_Localizacion", "Kilometraje_km"], inplace=True)
    
    # Asegurarse de que todos los valores en 'Lugar_de_inicio' sean cadenas
    estacionados_camion['Lugar_de_inicio'] = estacionados_camion['Lugar_de_inicio'].astype(str)

    # Obtener y mostrar los números de placa únicos para el mes correspondiente
    valores_unicos = estacionados_camion['Numero_de_placa'].unique()
    print(f"Números de placa únicos para {meses[i]}: {valores_unicos}")
    
    # Guardar los datos del mes actual en la lista total
    estacionados_camion_total.append(estacionados_camion)

# Concatenar los datos de todos los meses
estacionados_camion_total = pd.concat(estacionados_camion_total, ignore_index=True)

Números de placa únicos para Noviembre: ['AAW 4251' nan 'Número de placa del vehículo' 'Peugeot AVG9758' 'BYD1006'
 'CAA 1076' 'Wireless-16956']
Números de placa únicos para Diciembre: ['AAW 4251' nan 'Número de placa del vehículo' 'Peugeot AVG9758' 'BYD1006'
 'CAA 1076']
Números de placa únicos para Enero: ['AAW 4251' nan 'Número de placa del vehículo' 'Peugeot AVG9758' 'BYD1006'
 'CAA 1076']
Números de placa únicos para Febrero: ['AAW 4251' nan 'Número de placa del vehículo' 'Peugeot AVG9758' 'BYD1006'
 'CAA 1076']


In [6]:
estacionados_camion_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16338 entries, 0 to 16337
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Indice            16338 non-null  object        
 1   Numero_de_placa   16218 non-null  object        
 2   Estado_de_viaje   16218 non-null  object        
 3   Tiempo_de_Inicio  16202 non-null  datetime64[ns]
 4   Tiempo_Final      16202 non-null  datetime64[ns]
 5   Duracion          16218 non-null  object        
 6   Lugar_de_inicio   16338 non-null  object        
dtypes: datetime64[ns](2), object(5)
memory usage: 893.6+ KB


## Normalizacion de patentes y mapeo con los nombres requeridos

'CAA 1076' CONVERITDO EN 'BYD1004'

'Wireless-16956' CONVERITDO EN 'PartnerABG9758'

'AAW 4251' CONVERITDO EN 'PARTNER 4251'

In [7]:
# Normalizar las patentes y aplicar el mapeo
def normalizar_placas(df):
    df['Numero_de_placa'] = df['Numero_de_placa'].str.replace(' ', '').str.upper()
    return df

def aplicar_mapeo_placas(df, map_placas):
    df['Numero_de_placa'] = df['Numero_de_placa'].str.replace('|'.join(map_placas.keys()),
                                                        lambda x: map_placas[x], regex=True)
    return df

estacionados_camion_total = normalizar_placas(estacionados_camion_total)

map_placas = {
    'CAA 1076': 'BYD1004',
    'Wireless-16956': 'PartnerABG9758',
    'AAW 4251': 'PARTNER 4251'
}

def aplicar_mapeo_placas(df, map_placas):
    for key, value in map_placas.items():
        key_sin_espacios = key.replace(' ', '').upper()
        df['Numero_de_placa'] = df['Numero_de_placa'].str.replace(key_sin_espacios, value, case=False)
    return df

estacionados_camion_total = aplicar_mapeo_placas(estacionados_camion_total, map_placas)

# Eliminar filas con valores nulos y resetear el índice
estacionados_camion_total.dropna(subset=['Numero_de_placa', 'Tiempo_de_Inicio', 'Tiempo_Final'], inplace=True)


estacionados_camion_total.reset_index(drop=True, inplace=True)

#### Revisamos haber renombrado las placas a las que nos pidio Adrian:

In [8]:
# Verificar los cambios
estacionados_camion_total['Numero_de_placa'].unique()

array(['PARTNER 4251', 'PEUGEOTAVG9758', 'BYD1006', 'BYD1004',
       'PartnerABG9758'], dtype=object)

## Separar las Coordenadas camion_x camion_y

In [9]:
def verificar_coordenadas_camion(estacionados_camion):
    """
    Verifica las coordenadas de los camiones y separa las válidas de las inválidas.

    Parámetros:
    - estacionados_camion (pd.DataFrame): DataFrame con datos de camiones estacionados.

    Retorna:
    - coordenadas_camion_validas (pd.DataFrame): DataFrame con coordenadas válidas.
    - coordenadas_camion_invalidas (pd.DataFrame): DataFrame con coordenadas inválidas.
    """
    print("\nVerificando coordenadas en 'estacionados_camion'...")

    # Contar registros antes del filtrado
    total_registros_antes = len(estacionados_camion)

    # Filtrar coordenadas válidas
    coordenadas_camion_validas = estacionados_camion[
        estacionados_camion['camion_x'].between(-90, 90) & 
        estacionados_camion['camion_y'].between(-180, 180) & 
        estacionados_camion['camion_x'].notnull() & 
        estacionados_camion['camion_y'].notnull()
    ].copy()

    # Filtrar coordenadas inválidas
    coordenadas_camion_invalidas = estacionados_camion[
        ~(
            estacionados_camion['camion_x'].between(-90, 90) & 
            estacionados_camion['camion_y'].between(-180, 180) & 
            estacionados_camion['camion_x'].notnull() & 
            estacionados_camion['camion_y'].notnull()
        )
    ].copy()

    # Contar registros después del filtrado
    total_registros_despues = len(coordenadas_camion_validas)

    # Calcular eliminados y porcentaje
    eliminados = total_registros_antes - total_registros_despues
    porcentaje_eliminados = (eliminados / total_registros_antes) * 100 if total_registros_antes > 0 else 0

    # Imprimir resultados
    print(f"Total de camiones con coordenadas válidas: {total_registros_despues}")
    print(f"Total de camiones con coordenadas inválidas: {len(coordenadas_camion_invalidas)}")
    print(f"Total de camiones procesados: {total_registros_antes}")
    print(f"Registros eliminados: {eliminados} ({porcentaje_eliminados:.2f}%)")

    if not coordenadas_camion_invalidas.empty:
        print("\nCamiones con coordenadas inválidas:")
        print(coordenadas_camion_invalidas[['Numero_de_placa', 'camion_x', 'camion_y']])

        # Guardar registros inválidos en un archivo
        archivo_invalidos = "../Limpia/coordenadas_invalidas.xlsx"
        coordenadas_camion_invalidas.to_excel(archivo_invalidos, index=False)
        print(f"Registros de coordenadas inválidas guardados en: {archivo_invalidos}")

    return coordenadas_camion_validas, coordenadas_camion_invalidas

In [10]:
# AGREGADO PARA LOGGING
print("\n=== Iniciando Procesamiento de Coordenadas ===")
estacionados_antes_coords = estacionados_camion_total.copy()

# Función para convertir coordenadas
def convert_coord(coord):
    if coord is None:
        return None
    coord = coord.strip()
    if coord[-1] in ['N', 'E']:
        return float(coord[:-1])  # Devuelve el valor positivo
    elif coord[-1] in ['S', 'W']:
        return -float(coord[:-1])  # Devuelve el valor negativo
    else:
        try:
            return float(coord)  # Intenta convertir a float
        except ValueError:
            return None  # Retorna None si no se puede convertir

# Función para separar las coordenadas
def split_coords(lugar):
    try:
        lat, lon = lugar.split(',')
        return lat.strip(), lon.strip()
    except ValueError:
        return None, None

# Crear nuevas columnas 'latitud' y 'longitud'
estacionados_camion_total[['longitud', 'latitud']] = estacionados_camion_total['Lugar_de_inicio'].apply(lambda x: pd.Series(split_coords(x)))

# Convertir las coordenadas de 'latitud' y 'longitud' a numéricas
estacionados_camion_total['latitud'] = estacionados_camion_total['latitud'].apply(convert_coord)
estacionados_camion_total['longitud'] = estacionados_camion_total['longitud'].apply(convert_coord)

# Eliminar filas con valores nulos en latitud y longitud
estacionados_camion_total.dropna(subset=['latitud', 'longitud'], inplace=True)

# Renombrar columnas de coordenadas
estacionados_camion_total.rename(columns={'latitud': 'camion_y', 'longitud': 'camion_x'}, inplace=True)

# Filtrar filas con coordenadas válidas
estacionados_camion_total = estacionados_camion_total[
    estacionados_camion_total['camion_x'].between(-90, 90) & 
    estacionados_camion_total['camion_y'].between(-180, 180) &
    estacionados_camion_total['camion_x'].notnull() & 
    estacionados_camion_total['camion_y'].notnull()
]

# Verificar coordenadas
coordenadas_validas, coordenadas_invalidas = verificar_coordenadas_camion(estacionados_camion_total)


=== Iniciando Procesamiento de Coordenadas ===

Verificando coordenadas en 'estacionados_camion'...
Total de camiones con coordenadas válidas: 16202
Total de camiones con coordenadas inválidas: 0
Total de camiones procesados: 16202
Registros eliminados: 0 (0.00%)


### Procesamos duracion

In [11]:
# Antes de convertir duración
print("\n=== Iniciando Procesamiento de Duración ===")
estacionados_antes_duracion = estacionados_camion_total.copy()


# Función para convertir "XhYMinutosZs" a formato decimal
def convertir_a_decimal(duracion):
  try:
      duracion = str(duracion)
      # Manejar formato con días
      if 'd' in duracion:
          dias_parte = duracion.split('d')[0]
          resto = duracion.split('d')[1]
          dias = int(dias_parte)
          minutos_por_dias = dias * 24 * 60
      else:
          minutos_por_dias = 0
          resto = duracion

      # Manejar horas y minutos
      match = re.match(r'(?:(\d+)h)?(?:(\d+)Minutos)?(?:(\d+)s)?', resto)
      if match:
          horas = int(match.group(1)) if match.group(1) else 0
          minutos = int(match.group(2)) if match.group(2) else 0
          segundos = int(match.group(3)) if match.group(3) else 0

          total_minutos = minutos_por_dias + (horas * 60) + minutos + (segundos / 60)
          return round(total_minutos, 2)
  except:
      return None
# Aplicar la función a la columna 'Duracion'
estacionados_camion_total['Duracion'] = estacionados_camion_total['Duracion'].apply(convertir_a_decimal)

# Eliminar filas con Duracion nula
estacionados_camion_total.dropna(subset=['Duracion'], inplace=True)


# Registramos el proceso
log_proceso(estacionados_antes_coords, estacionados_camion_total, "Coordenadas")


=== Iniciando Procesamiento de Duración ===

=== Etapa: Coordenadas ===
Registros antes: 16202
Registros después: 16202
Registros eliminados: 0 (0.00%)


In [12]:
# Después de procesar duración y eliminar nulos
log_proceso(estacionados_antes_duracion, estacionados_camion_total, "Duración")


=== Etapa: Duración ===
Registros antes: 16202
Registros después: 16202
Registros eliminados: 0 (0.00%)


## Filtro rango por hora, sacamos camiones mas de 18 a 7 am

In [13]:
print("\n=== Iniciando Filtrado por Horario ===")
estacionados_antes_horario = estacionados_camion_total.copy()


# Contar registros previos al tratamiento
registros_previos = len(estacionados_camion_total)

# Filtrar los registros donde la hora esté entre las 18:00 y las 07:00
estacionados_camion_total = estacionados_camion_total[~((estacionados_camion_total['Tiempo_de_Inicio'].dt.hour >= 18) | 
                                                     (estacionados_camion_total['Tiempo_de_Inicio'].dt.hour < 7))]
# Contar registros después del tratamiento
registros_post_tratamiento = len(estacionados_camion_total)

# Imprimir los resultados
print(f"Registros previos al tratamiento: {registros_previos}")
print(f"Registros después del tratamiento: {registros_post_tratamiento}")
	
 
 # Logging del filtrado por hora
log_proceso(estacionados_antes_horario, estacionados_camion_total, "Filtrado_Horario")	


=== Iniciando Filtrado por Horario ===
Registros previos al tratamiento: 16202
Registros después del tratamiento: 15199

=== Etapa: Filtrado_Horario ===
Registros antes: 16202
Registros después: 15199
Registros eliminados: 1003 (6.19%)
Registros eliminados guardados en: ../Limpia/eliminados_filtrado_horario.xlsx


In [14]:
valores_unicos = estacionados_camion_total['Numero_de_placa'].unique()
# Imprimir los valores únicos
print(valores_unicos)

['PARTNER 4251' 'PEUGEOTAVG9758' 'BYD1006' 'BYD1004' 'PartnerABG9758']


In [15]:
# Antes de crear los dos datasets finales
print("\n=== Resumen Final del Procesamiento ===")
print(f"Registros iniciales totales: {sum(len(df) for df in lista_tracks)}")
print(f"Registros finales: {len(estacionados_camion_total)}")
print(f"Registros eliminados totales: {sum(len(df) for df in lista_tracks) - len(estacionados_camion_total)}")


=== Resumen Final del Procesamiento ===
Registros iniciales totales: 16338
Registros finales: 15199
Registros eliminados totales: 1139


### Antes de guardar, crear 2 dataset, uno agregando la condicion CONDUCIR (para mostrar en Proyectos Clientes)

In [16]:
# Antes de guardar, crear dos datasets
estacionados_camion = estacionados_camion_total[
    estacionados_camion_total["Estado_de_viaje"].isin(["Estacionamiento", "Parking"])
].copy()


todos_estados_camion = estacionados_camion_total.copy()

# Guardar ambos archivos
estacionados_camion.to_excel("../Limpia/estacionados_camion.xlsx", index=False)

# Decidimos dejar de usarlo asi que lo comento.
# todos_estados_camion.to_excel("../Limpia/estacionados_camion_todos_estados.xlsx", index=False)

In [17]:
estacionados_camion.info()
estacionados_camion.tail(2)
# estacionados_camion.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 7600 entries, 1 to 16201
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Indice            7600 non-null   object        
 1   Numero_de_placa   7600 non-null   object        
 2   Estado_de_viaje   7600 non-null   object        
 3   Tiempo_de_Inicio  7600 non-null   datetime64[ns]
 4   Tiempo_Final      7600 non-null   datetime64[ns]
 5   Duracion          7600 non-null   float64       
 6   Lugar_de_inicio   7600 non-null   object        
 7   camion_x          7600 non-null   float64       
 8   camion_y          7600 non-null   float64       
dtypes: datetime64[ns](2), float64(3), object(4)
memory usage: 593.8+ KB


,Indice,Numero_de_placa,Estado_de_viaje,Tiempo_de_Inicio,Tiempo_Final,Duracion,Lugar_de_inicio,camion_x,camion_y
16199,384,BYD1004,Estacionamiento,2025-02-12 10:15:49,2025-02-12 10:27:02,11.22,"34.746902S,55.768097W",-34.746902,-55.768097
16201,386,BYD1004,Estacionamiento,2025-02-12 10:28:16,2025-02-12 10:34:15,5.98,"34.743627S,55.765974W",-34.743627,-55.765974


In [18]:
todos_estados_camion.info()
todos_estados_camion.tail(5)

<class 'pandas.core.frame.DataFrame'>
Index: 15199 entries, 0 to 16201
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Indice            15199 non-null  object        
 1   Numero_de_placa   15199 non-null  object        
 2   Estado_de_viaje   15199 non-null  object        
 3   Tiempo_de_Inicio  15199 non-null  datetime64[ns]
 4   Tiempo_Final      15199 non-null  datetime64[ns]
 5   Duracion          15199 non-null  float64       
 6   Lugar_de_inicio   15199 non-null  object        
 7   camion_x          15199 non-null  float64       
 8   camion_y          15199 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(4)
memory usage: 1.2+ MB


,Indice,Numero_de_placa,Estado_de_viaje,Tiempo_de_Inicio,Tiempo_Final,Duracion,Lugar_de_inicio,camion_x,camion_y
16197,382,BYD1004,Estacionamiento,2025-02-12 09:47:17,2025-02-12 10:10:51,23.57,"34.740957S,55.764235W",-34.740957,-55.764235
16198,383,BYD1004,Conducir,2025-02-12 10:10:51,2025-02-12 10:15:49,4.97,"34.741717S,55.763615W",-34.741717,-55.763615
16199,384,BYD1004,Estacionamiento,2025-02-12 10:15:49,2025-02-12 10:27:02,11.22,"34.746902S,55.768097W",-34.746902,-55.768097
16200,385,BYD1004,Conducir,2025-02-12 10:27:02,2025-02-12 10:28:16,1.23,"34.747104S,55.769010W",-34.747104,-55.769010
16201,386,BYD1004,Estacionamiento,2025-02-12 10:28:16,2025-02-12 10:34:15,5.98,"34.743627S,55.765974W",-34.743627,-55.765974
